## Installation

In [5]:
!pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/b3/38/5aac23e57d61707f91914506902e621632de8dc56b60446459901469b9e2/selenium-4.24.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/1c/70/efa56ce2271c44a7f4f43533a0477e6854a0948e9f7b76491de1fd3be7c9/trio-0.26.2-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions~=4.9 from https://files.pythonhosted.org/packages/26/9f/ad63fc0248c5379346306f8668cda6e2e2e9c95e01216d2b8ffd9ff037d0/typing_extensions-4.12.2-py3-none-any.whl.metadata
  Obtaining dependency information for websocket-client~=1.8 from https://files.pythonhosted.org/packages/5a/84/44687a29792a70e111c5c477230a72c4b957d88d16

## Importing

In [3]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

## Question 1

Q1: In this question you have to scrape data using the filters available on the webpage You have to use the location and 
salary filter. 
You have to scrape data for “Data Scientist” designation for first 10 job results. 
You have to scrape the job-title, job-location, company name, experience required. 
The location filter to be used is “Delhi/NCR”. The salary filter to be used is “3-6” lakhs 
The task will be done as shown in the below steps: 
1. first get the web page https://www.naukri.com/
2. Enter “Data Scientist” in “Skill, Designations, and Companies” field. 
3. Then click the search button. 
4. Then apply the location filter and salary filter by checking the respective boxes 
5. Then scrape the data for the first 10 jobs results you get. 
6. Finally create a dataframe of the scraped data. 
Note: All of the above steps have to be done in code. No step is to be done manually.

In [142]:
driver = webdriver.Chrome()
driver.get('https://www.naukri.com/')

#time.sleep(5)
#waits untill the page loads
driver.implicitly_wait(2)
# Enter “Data Scientist” in “Skill, Designations, and Companies” field and send it
job_title_input = driver.find_element(By.XPATH, '//*[@id="root"]/div[7]/div/div/div[1]/div/div/div/div[1]/div/input')
job_title_input.send_keys("Data Scientist")

# Get search button and send these details by clicking it
search_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[7]/div/div/div[6]')
search_button.click()

time.sleep(10)
driver.quit()

In [5]:
# some times working and some times I am getting the below error in the automated browser 
#This site can’t be reached
#The webpage at https://www.naukri.com/data-scientist-jobs?k=data%20scientist might be temporarily down or it may have moved permanently to a new web address.
def scrape_naukri_jobs():
    #initiate chrome driver and navigate to url
    driver = webdriver.Chrome()
    driver.get('https://www.naukri.com/')

    #waits untill the page loads
    driver.implicitly_wait(10)

    # Enter “Data Scientist” in “Skill, Designations, and Companies” field and send it
    job_title_input = driver.find_element(By.XPATH, '//*[@id="root"]/div[7]/div/div/div[1]/div/div/div/div[1]/div/input')
    job_title_input.send_keys("Data Scientist")

    # Get search button and send these details by clicking it
    search_button = driver.find_element(By.XPATH, '//*[@id="root"]/div[7]/div/div/div[6]')
    search_button.click()

    #location filter
    location_filter = driver.find_element(By.XPATH, '//*[@id="search-result-container"]/div[1]/div[1]/div/div/div[2]/div[4]/div[2]/div[3]/label/p/span[1]')
    #scroll to the  element location filter to “Delhi/NCR”
    actions = ActionChains(driver)
    actions.move_to_element(location_filter).perform()
    time.sleep(1)
    location_filter.click()
    time.sleep(1)
    
    #salary filter
    salary_filter = driver.find_element(By.XPATH, '//*[@id="search-result-container"]/div[1]/div[1]/div/div/div[2]/div[5]/div[2]/div[2]/label/p/span[1]')
    #scroll to the  element salary filter to “3-6” lakhs"
    actions.move_to_element(salary_filter).perform()
    time.sleep(1)
    salary_filter.click()
    time.sleep(1)

    # scrape the first ten jobs
    jobs = driver.find_elements(By.XPATH, './/div[@class="cust-job-tuple layout-wrapper lay-2 sjw__tuple "]')[:10] #cust-job-tuple layout-wrapper lay-2 sjw__tuple 

        
    job_titles = []
    job_locations = [] 
    company_names = []
    experience_required = []
    
    #looping through jobs to fetch the details
    for job in jobs:
        #scroll to the job element
        actions = ActionChains(driver)
        actions.move_to_element(job).perform()

        time.sleep(0.5)

        #find the job details
        job_title = job.find_element(By.XPATH, './/a[@class="title "]').text.strip() #//*[@id="listContainer"]/div[2]/div/div[1]/div/div[1]/a
        company_name = job.find_element(By.XPATH, './/div[@class=" row2"]/span/a').text.strip() #//*[@id="listContainer"]/div[2]/div/div[1]/div/div[2]/span/a[1]
        experience = job.find_element(By.XPATH, './/span[@class="expwdth"]').text.strip() #//*[@id="listContainer"]/div[2]/div/div[1]/div/div[3]/div/span[1]/span/span
        #job_location = job.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div/div[3]/div[1]/a').text.strip()
        
        #to check and get if multiple locations are present
        hover_element = job.find_element(By.XPATH, './/span[@class="locWdth"]') #//*[@id="listContainer"]/div[2]/div/div[1]/div/div[3]/div/span[3]/span/span
        actions.move_to_element(hover_element).perform()
        time.sleep(0.5)
        job_location = hover_element.text
        
        #append all job details to the list
        job_titles.append(job_title)
        company_names.append(company_name)
        experience_required.append(experience)
        job_locations.append(job_location)
        
        print(job_title, company_name, experience, job_location)
    #time.sleep(160)
    #close the driver
    driver.quit()

    #create a dataframe from the scraped data
    jobs_df = pd.DataFrame({
        'Job Title': job_titles,
        'Job Location': job_locations,
        'Company Name': company_names,
        'Experience': experience_required
    })

     # Keeping pandas from truncating long strings
    pd.set_option('display.max_colwidth', None)
    pd.reset_option('display.width')
    print(jobs_df)
    
scrape_naukri_jobs()



Data Scientist Nityo Infotech 3-7 Yrs Kolkata, Mumbai, New Delhi, Hyderabad, Pune, Chennai, Bengaluru
Data Scientist (AI/ML)|| US Based MNC || Noida US based Software Company 3-5 Yrs Hybrid - Noida
Data Scientist Collegedunia 0-2 Yrs Gurugram
Data Scientist Sociomix 0-5 Yrs New Delhi
Data scientist Growthjockey 0-1 Yrs Gurugram
Data Scientist/Data Analyst - LLM Hexaconcepts 2-6 Yrs Kolkata, Mumbai, New Delhi, Hyderabad, Pune, Chennai, Bengaluru
Data Scientist Biz Tech Consultants 3-8 Yrs Gurugram
Data Scientist Biz Tech Consultants 3-8 Yrs Ghaziabad
Data Scientist Biz Tech Consultants 3-8 Yrs Noida
Data Scientist (Telco) PayU 2-7 Yrs Gurugram, Bengaluru
                                        Job Title  \
0                                  Data Scientist   
1  Data Scientist (AI/ML)|| US Based MNC || Noida   
2                                  Data Scientist   
3                                  Data Scientist   
4                                  Data scientist   
5               Data

## Question 2

Q2: Write a python program to scrape data for “Data Scientist” Job position in “Bangalore” location. You have to scrape the 
job-title, job-location, company_name, experience_required. You have to scrape first 10 jobs data.
This task will be done in following steps:
1. First get the webpage https://www.shine.com/
2. Enter “Data Analyst” in “Job title, Skills” field and enter “Bangalore” in “enter the location” field.
3. Then click the searchbutton. 
4. Then scrape the data for the first 10 jobs results you get. 
5. Finally create a dataframe of the scraped data. 

In [83]:
def scrapeTenjobs():
    #initiate chrome driver and navigate to url
    driver = webdriver.Chrome()
    driver.get('https://www.shine.com/')
    
    #waits untill the page loads
    driver.implicitly_wait(2)
    
    # Get the search bar , click it to enter the skill, location
    searchBar_field = driver.find_element(By.XPATH, '//*[@id="webSearchBar"]/input')
    searchBar_field.click()

    # Enter “Data Analyst” in “Job title, Skills” field and send it
    job_title_field = driver.find_element(By.XPATH, '//*[@id="id_q"]')
    job_title_field.send_keys("Data Analyst")

    # Enter “Bangalore” in “Location” field and send it
    location_field = driver.find_element(By.XPATH, '//*[@id="id_loc"]')
    location_field.send_keys("Bangalore")

    # Get search button and send these details by clicking it
    search_button = driver.find_element(By.XPATH, '//*[@id="__next"]/div[4]/div/div[2]/div[2]/div/form/div/div[2]/div/button')
    search_button.click()

    # scrape the first ten jobs
    jobs = driver.find_elements(By.CLASS_NAME, 'jobCard_jobCard__jjUmu')[:10]
    
    job_titles = []
    job_locations = [] 
    company_names = []
    experience_required = []
    
    #looping through jobs to fetch the details
    for job in jobs:
        #scroll to the job element
        actions = ActionChains(driver)
        actions.move_to_element(job).perform()

        #clicking the scrolled job and waiting to display its content
        job.click()
        time.sleep(0.5)

        #find the job details
        job_title = job.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div/a/p').text.strip()
        company_name = job.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div/div[2]/span').text.strip()
        experience = job.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div/div[3]/div[2]').text.strip()
        job_location = job.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div/div[3]/div[1]/a').text.strip()
        
        #to check and get if multiple locations are present
        hover_element = job.find_element(By.XPATH, '//*[@id="__next"]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div/div[3]/div[1]/div')
        actions.move_to_element(hover_element).perform()
        multiple_locations = job.find_elements(By.XPATH, '//*[@id="__next"]/div[2]/div[4]/div/div[1]/div[1]/div[2]/div/div[3]/div[1]/div/div/a')
        for location in multiple_locations:
            job_location += "," + location.text

        #append all job details to the list
        job_titles.append(job_title)
        company_names.append(company_name)
        experience_required.append(experience)
        job_locations.append(job_location)
        
        #print(job_title, company_name, experience, job_location)
        
    #close the driver
    driver.quit()
    
    #print(f"{job_titles}\n\n{company_names}\n\n{experience_required}\n\n{job_locations}\n\n")
    
    #create a dataframe from the scraped data
    jobs_df = pd.DataFrame({
        'Job Title': job_titles,
        'Job Location': job_locations,
        'Company Name': company_names,
        'Experience': experience_required
    })

     # Keeping pandas from truncating long strings
    pd.set_option('display.max_colwidth', None)
    pd.reset_option('display.width')
    print(jobs_df)
    
scrapeTenjobs()

                                                       Job Title  \
0  Data Analyst , Senior Data Analyst , Data Analyst - Tech Lead   
1                                                   Data Analyst   
2                Risk Data Analyst (Senior Quantitative Analyst)   
3                                        Healthcare Data Analyst   
4                                            Senior Data Analyst   
5                                 Urgent Hiring For Data Analyst   
6                               Data Catalog with Data Goverance   
7                                        Data Governance Analyst   
8                                             Healthcare Analyst   
9                                               Business Analyst   

                                                                          Job Location  \
0            Bangalore,Chennai,Noida,Hyderabad,Gurugram,Kolkata,Pune,Mumbai City,Delhi   
1                                             Bangalore,Kolkata,Pune,Mu

## Question 3

Q3: Scrape 100 reviews data from flipkart.com for iphone11 phone. You have to go the link: https://www.flipkart.com/apple-iphone-11-black-64-gb/product-reviews/itm4e5041ba101fd?pid=MOBFWQ6BXGJCEYNY&lid=LSTMOBFWQ6BXGJCEYNYZXSHRJ&marketplace=FLIPKART

As shown in the above page you have to scrape the tick marked attributes. These are:
1. Rating
2. Review summary
3. Full review
4. You have to scrape this data for first 100reviews.
Note: All the stepsrequired during scraping should be done through code only and not manually.ews.T


In [257]:
def scrape_flipkart_iphone11_reviews():
     #initiate chrome driver and navigate to url
    driver = webdriver.Chrome()
    driver.get('https://www.flipkart.com/apple-iphone-11-black-64-gb/product-reviews/itm4e5041ba101fd?pid=MOBFWQ6BXGJCEYNY&lid=LSTMOBFWQ6BXGJCEYNYZXSHRJ&marketplace=FLIPKART')
    
    #waits untill the page loads
    driver.implicitly_wait(10)
    
    all_reviews = []
    page_number = 2
    
    while len(all_reviews) < 100:
        #scraping all reviews of current page
        reviews = driver.find_elements(By.CLASS_NAME, 'EKFha-')
    
        for review in reviews:
             # Scroll to the review element using JavaScript Executor
            driver.execute_script("arguments[0].scrollIntoView()", review)
            time.sleep(1)
            
            rating = review.find_element(By.XPATH, './/div[@class="XQDdHH Ga3i8K"]').text  #'//*[@id="container"]/div/div[3]/div/div/div[2]/div[3]/div/div/div/div[1]/div').text
            review_summary = review.find_element(By.XPATH, './/p[@class="z9E0IG"]').text  #'//*[@id="container"]/div/div[3]/div/div/div[2]/div[3]/div/div/div/div[1]/p').text
            full_review = review.find_element(By.XPATH, './/div[@class="ZmyHeo"]').text #'//*[@id="container"]/div/div[3]/div/div/div[2]/div[3]/div/div/div/div[2]/div/div/div').text
            
            all_reviews.append([rating, review_summary,full_review])
            if len(all_reviews) == 100:
                break
            time.sleep(1)
            
        #navigating to next page untill 100 reviews are found
        next_button = driver.find_element(By.XPATH, f"//*[@id='container']/div/div[3]/div/div/div[2]/div[13]/div/div/nav/a[contains(text(), {page_number})]")
        next_button.click()
        page_number += 1
        
        #wait to load the page
        time.sleep(2)
     #close the driver   
    driver.quit()

    print(f"Total reviews count: {len(all_reviews)}\n")
    print(f"     All Reviews\n")
    for review in all_reviews:
        print("Rating:",review[0])
        print("Review Summary:",review[1])
        print("Full Review:",review[2])
        print()
        
scrape_flipkart_iphone11_reviews()

Total reviews count: 100

     All Reviews

Rating: 5
Review Summary: Best in the market!
Full Review: Good Camera

Rating: 5
Review Summary: Terrific
Full Review: Very very good

Rating: 5
Review Summary: Classy product
Full Review: Camera is awesome
Best battery backup
A performer 👌🏻
It will be a real value for money if they provide charger and earphone inbox!

Rating: 5
Review Summary: Worth every penny
Full Review: Feeling awesome after getting the delivery of my phone. Dual speakers makes the sound quality very loud. Amazing!!

Rating: 5
Review Summary: Perfect product!
Full Review: Photos super

Rating: 5
Review Summary: Terrific purchase
Full Review: Value for money 😍

Rating: 5
Review Summary: Wonderful
Full Review: This is amazing at all

Rating: 5
Review Summary: Excellent
Full Review: NYC

Rating: 5
Review Summary: Brilliant
Full Review: very good camera quality

Rating: 5
Review Summary: Perfect product!
Full Review: V Good all

Rating: 5
Review Summary: Awesome
Full Review

## Question 4

Q4: Scrape data forfirst 100 sneakers you find whenyouvisitflipkart.com and search for “sneakers” inthe search
field.
You have to scrape 3 attributes of each sneaker:
1. Brand
2. ProductDescription
3. Price
As shown in the below image, you have to scrape the above attributes.


In [140]:
def scrape_flipkart_sneakers_details():
     #initiate chrome driver and navigate to url
    driver = webdriver.Chrome()
    driver.get('https://www.flipkart.com')
    
    #waits untill the page loads
    driver.implicitly_wait(10)

    # search for “sneakers” in the search field
    searchBar_field = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[1]/div/div/div/div/div[1]/div/div/div/div[1]/div[1]/header/div[1]/div[2]/form/div/div/input')
    searchBar_field.send_keys("sneakers")

    # click for the search
    search_field = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[1]/div/div/div/div/div[1]/div/div/div/div[1]/div[1]/header/div[1]/div[2]/form/div/button')
    search_field.click()

    all_sneakers_details = []
    page_number = 2
    
    while len(all_sneakers_details) < 100:
        #scraping all sneakers of current page
        sneakers = driver.find_elements(By.CLASS_NAME, '_1sdMkc') #_1sdMkc LFEi7Z
        for sneaker in sneakers:
             # Scroll to the review element using JavaScript Executor
            driver.execute_script("arguments[0].scrollIntoView()", sneaker)
            time.sleep(1)
            
            brand = sneaker.find_element(By.XPATH, './/div[@class="syl9yP"]').text 
            price = sneaker.find_element(By.XPATH, './/div[@class="Nx9bqj"]').text
            
            # Click on the product link to navigate to the product page
            product_description_link = sneaker.find_element(By.CLASS_NAME, 'WKTcLC').get_attribute('href')
            driver.execute_script("window.open()") 
            #time.sleep(1)

            #switching window to product page
            driver.switch_to.window(driver.window_handles[1]) 
            driver.get(product_description_link)
            time.sleep(1)
            
            product_description = driver.find_element(By.XPATH, '//*[@id="container"]/div/div[3]/div[1]/div[2]/div[2]/div/div[1]/h1/span[2]').text

            #Close the new window and switch to all sneakers window
            driver.execute_script("window.close()") 
            driver.switch_to.window(driver.window_handles[0]) 

            all_sneakers_details.append([brand, product_description, price])
            if len(all_sneakers_details) == 100:
                break
            #time.sleep(1)
            
        #navigating to next page untill 100 reviews are found
        next_button = driver.find_element(By.XPATH, f'//*[@id="container"]/div/div[3]/div[1]/div[2]/div[12]/div/div/nav/a[contains(text(), {page_number})]')
        next_button.click()
        page_number += 1
        
        #wait to load the page
        time.sleep(10)
     #close the driver   
    driver.quit()

    print(f"Total Sneakers count: {len(all_sneakers_details)}\n")
    print(f"   Sneakrs\n")
    for sneaker in all_sneakers_details:
        print("Brand:",sneaker[0])
        print("Product Description:",sneaker[1])
        print("Price:",sneaker[2])
        print()
        
scrape_flipkart_sneakers_details()

Total Sneakers count: 100

   Sneakrs

Brand: Sparx
Product Description: SM 671 | Stylish, Comfortable | Sneakers For Men  (Black , 8)
Price: ₹974

Brand: Sparx
Product Description: SM 852 | Stylish, Comfortable | Sneakers For Men  (Grey, Orange , 8)
Price: ₹879

Brand: WROGN
Product Description: Sneakers For Men  (Blue , 9)
Price: ₹699

Brand: WROGN
Product Description: Sneakers For Men  (Navy , 11)
Price: ₹909

Brand: Vellinto
Product Description: Casual Sneakrs For Men Sneakers For Men  (Green, Khaki , 8)
Price: ₹599

Brand: Sparx
Product Description: SM 162 | Stylish, Comfortable | Sneakers For Men  (Navy , 7)
Price: ₹824

Brand: WROGN
Product Description: Sneakers For Men  (White , 11)
Price: ₹1,049

Brand: WROGN
Product Description: Sneakers For Men  (Multicolor , 6)
Price: ₹734

Brand: Roadster
Product Description: Casual Sneakers For Men  (Black, White , 8)
Price: ₹1,403

Brand: FLYING MACHINE
Product Description: CARPE 2.0 Sneakers For Men  (Multicolor , 11)
Price: ₹1,199

Bra

## Question 5

Q5: Go to webpage https://www.amazon.in/ Enter “Laptop” in the search field and then click the search icon. Then set CPU
Type filter to “Intel Core i7” as shown in the below imag
Aftersetting the filters scrape first 10 laptops data. You have to scrape 3 attributes for each laptop:
1. Title
2. Ratings
3. Pricee

In [43]:
def scrape_amazon_laptop_details():
     #initiate chrome driver and navigate to url
    driver = webdriver.Chrome()
    driver.get('https://www.amazon.in/')
    
    #waits untill the page loads
    driver.implicitly_wait(10)

    # Get the search bar , click it to enter the skill, location
    searchBar_field = driver.find_element(By.XPATH, '//*[@id="twotabsearchtextbox"]')
    searchBar_field.click()

    # search for “Laptop” in the search field
    searchBar_field.send_keys("Laptop")

    # click for the search
    search_icon = driver.find_element(By.XPATH, '//*[@id="nav-search-submit-button"]')
    search_icon.click()

    cpu_filter = driver.find_element(By.XPATH, '//*[@id="p_n_feature_thirteen_browse-bin/12598163031"]/span/a/span')
    #scroll to the  element CPU Type filter to “Intel Core i7
    actions = ActionChains(driver)
    actions.move_to_element(cpu_filter).perform()
    time.sleep(1)

    cpu_filter.click()

    # scrape the first ten filtered laptops
    laptops = driver.find_elements(By.XPATH, '//*[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]')[:10]

    all_laptops=[]
    for laptop in laptops:
        # Scroll to the laptop element using JavaScript Executor
        driver.execute_script("arguments[0].scrollIntoView()", laptop)
        time.sleep(1)
        
        # Click on the laptop title link to navigate to the laptop product page
        product_title_link = laptop.get_attribute('href')
        driver.execute_script("window.open()") 
        #time.sleep(1)
    
        #switching window to product page
        driver.switch_to.window(driver.window_handles[1]) 
        driver.get(product_title_link)
        time.sleep(1)
        
        product_title = driver.find_element(By.XPATH, '//*[@id="productTitle"]').text
        
        #some laptops doesn't have prices or ratings to handle this
        ratings = driver.find_elements(By.XPATH, '//*[@id="acrPopover"]/span[1]/a/span')
        prices = driver.find_elements(By.XPATH, '//*//*[@id="corePriceDisplay_desktop_feature_div"]/div[1]/span[3]/span[2]/span[2]')
        rating = ""
        price = ""
        
        if ratings != []:
            rating = ratings[0].text
        else:
            rating = "None"

        if prices != []:
            price = prices[0].text
        else:
            price = "Not Mentioned"
    
        #Close the new window and switch to all sneakers window
        driver.execute_script("window.close()") 
        driver.switch_to.window(driver.window_handles[0]) 
        
        all_laptops.append({
            'Title': product_title,
            'Rating': rating,
            'Price': price,
        })
        #print(len(all_laptops),product_title,rating,price,'\n')
        #time.sleep(5)

    #close the driver   
    driver.quit()

    print(f"Total Laptops count: {len(all_laptops)}\n")
    print(f"   Laptops    \n")
    for laptop in all_laptops:
        print(laptop)
        print()
        
scrape_amazon_laptop_details()

Total Laptops count: 10

   Laptops    

{'Title': 'HP Laptop 15s, 12th Gen Intel Core i7-1255U, 15.6-inch (39.6 cm), FHD, 16GB DDR4, 512GB SSD, Intel Iris Xe graphics, Backlit KB, Thin & light (Win 11, MSO 2021, Silver, 1.69 kg), fq5190TU', 'Rating': '3.6', 'Price': '59,990'}

{'Title': 'Dell [Smartchoice] Inspiron 5430 Thin & Light Laptop, 13th Gen Intel Core i7-1360P/16GB/1TB SSD/14.0" (35.56cm) FHD+ WVA 250 nits/Backlit KB + FPR/Windows 11 + MSO\'21/15 Month McAfee/Silver/1.59kg', 'Rating': '3.3', 'Price': '75,490'}

{'Title': 'HP Pavilion 14 12th Gen Intel Core i7 16GB SDRAM/1TB SSD 14 inch(35.6cm) FHD,IPS,Micro-Edge Display/Intel Iris Xe Graphics/B&O/Win 11/Backlit KB/FPR/MSO 2021/Natural Silver,14-dv2015TU', 'Rating': '4.1', 'Price': '76,990'}

{'Title': 'Acer Aspire Lite 12th Gen Intel Core i7-1255U Premium Metal Laptop (Windows 11 Home/16 GB RAM/512 GB SSD) AL15-52, 39.62cm (15.6") Full HD Display, Metal Body, Steel Gray, 1.59 Kg', 'Rating': '3.6', 'Price': '49,990'}

{'Title'

## Question 6

Q6: Write a python program to scrape data for Top 1000 Quotes of All Time.
The above task will be done in following steps:
1. First get the webpagehttps://www.azquotes.com/
2. Click on TopQuote
3. Than scrap a)Quote b) Author c) Type Of Quotes

In [156]:
def scrape_Top_1000_quotes():
     #initiate chrome driver and navigate to url
    driver = webdriver.Chrome()
    driver.get('https://www.azquotes.com/')
    
    #waits untill the page loads
    driver.implicitly_wait(2)

    top_quotes = driver.find_element(By.XPATH, '//*[@id="menu"]/div/div[3]/ul/li[5]/a')
    top_quotes.click()

    all_quotes = []
    page_number = 2
    
    while len(all_quotes) < 1000:
        #scraping all quotes of current page
        quotes = driver.find_elements(By.XPATH, '//ul[@class="list-quotes"]/li')
        
        for quote in quotes:
             # Scroll to the quote element using JavaScript Executor
            actions = ActionChains(driver)
            actions.move_to_element(quote).perform()
            #time.sleep(1)
            
            quote_title = quote.find_element(By.XPATH, './/a[@class="title"]').text 
            author = quote.find_element(By.XPATH, './/div[@class="author"]/a').text 
            types_of_quotes = quote.find_elements(By.XPATH, './/div[@class="tags"]/a')
            types_of_quotes_texts = []
            for quote_type in types_of_quotes:
                types_of_quotes_texts.append(quote_type.text)
           
            type_of_quote = ",".join(types_of_quotes_texts)

            all_quotes.append([quote_title, author, type_of_quote])
            print(f"Quote{len(all_quotes)}\nQuote: {quote_title}\nAuthor: {author}\nType Of Quote: {type_of_quote}\n")
            if len(all_quotes) == 1000:
                break
            #time.sleep(1)
            
        #navigating to next page untill 1000 quotes are found
        try:
            next_button = WebDriverWait(driver, 1).until(
                EC.element_to_be_clickable((By.LINK_TEXT, "Next →"))
            )
            actions.move_to_element(next_button).perform()
            next_button.click()
        except TimeoutException:
            break
 
    #close the driver   
    driver.quit()
    
    print(f"Total Top Quotes count: {len(all_quotes)}\n")
    print(f"     1000 Top Quotes\n")
    for quote in all_quotes:
        print("Quote:",quote[0])
        print("Author:",quote[1])
        print("Type of Quote:",quote[2])
        print()

scrape_Top_1000_quotes()

Quote1
Quote: The essence of strategy is choosing what not to do.
Author: Michael Porter
Type Of Quote: Essence,Deep Thought,Transcendentalism

Quote2
Quote: One cannot and must not try to erase the past merely because it does not fit the present.
Author: Golda Meir
Type Of Quote: Inspiration,Past,Trying

Quote3
Quote: Patriotism means to stand by the country. It does not mean to stand by the president.
Author: Theodore Roosevelt
Type Of Quote: Country,Peace,War

Quote4
Quote: Death is something inevitable. When a man has done what he considers to be his duty to his people and his country, he can rest in peace. I believe I have made that effort and that is, therefore, why I will sleep for the eternity.
Author: Nelson Mandela
Type Of Quote: Inspirational,Motivational,Death

Quote5
Quote: You have to love a nation that celebrates its independence every July 4, not with a parade of guns, tanks, and soldiers who file by the White House in a show of strength and muscle, but with family picn

NameError: name 'TimeoutException' is not defined

## Question 7

Q7: Write a python program to display list of respected former Prime Ministers of India (i.e. Name,
Born-Dead, Term of office, Remarks) from https://www.jagranjosh.com/general-knowledge/list-ofall-prime-ministers-of-india-1473165149-1
scrap the mentioned data and make the DataFrame

In [73]:
def scrape_primeMinister_details():
     #initiate chrome driver and navigate to url
    driver = webdriver.Chrome()
    driver.get('https://www.jagranjosh.com/general-knowledge/list-ofall-prime-ministers-of-india-1473165149-1')
    
    #waits untill the page loads
    driver.implicitly_wait(10)

    time.sleep(60)
    #close the driver   
    driver.quit()

scrape_primeMinister_details()

## Question 8

Q8: Write a python program to display list of 50 Most expensive cars in the world
(i.e. Car name and Price) from https://www.motor1.com/
This task will be done in following steps:
1. First get the webpage https://www.motor1.com/
2. Then You have to type in the search bar ’50 most expensive cars’
3. Then click on 50 most expensive carsin the world..
4. Then scrap thementioned data and make the dataframe

In [119]:
def scrape_50_mostExpensive_cars():
     #initiate chrome driver and navigate to url
    driver = webdriver.Chrome()
    driver.get('https://www.motor1.com/')
    
    #waits untill the page loads
    driver.implicitly_wait(10)

    time.sleep(60)
    #close the driver   
    driver.quit()

scrape_50_mostExpensive_cars()